Import Required Libraries

In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score


In [3]:
# Build dataset from folders
def build_dataframe():
    data = []
    for label, folder in enumerate(['ai', 'real']):
        n = 0
        for file in os.listdir(folder):
            if n == 20000:
                break
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.webp', '.gif')):
                n += 1
                data.append({'file_name': os.path.join(folder, file), 'label': label})
    df = pd.DataFrame(data)
    return train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

train_df, val_df = build_dataframe()
test_files = os.listdir('test_data/teamspace/studios/this_studio/final_test_renamed')
test_df = pd.DataFrame({'file_name': [f'test_data/teamspace/studios/this_studio/final_test_renamed/{x}' for x in test_files]})


In [4]:
# Dataset
class ImageDataset(Dataset):
    def __init__(self, df, transform=None, is_test=False):
        self.df = df
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.iloc[idx]['file_name']
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.is_test:
            return image, -1
        else:
            label = int(self.df.iloc[idx]['label'])
            return image, label

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [5]:

# Hyperparameters from Optuna
BATCH_SIZE = 64
LR = 0.000762
WEIGHT_DECAY = 0.0003804

# DataLoaders (⬅️ Made memory-efficient: num_workers=2 and pin_memory=True)
train_dataset = ImageDataset(train_df, transform=train_transform)
val_dataset = ImageDataset(val_df, transform=test_transform)
test_dataset = ImageDataset(test_df, transform=test_transform, is_test=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [7]:
# Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = efficientnet_b0(weights=None)  # ⬅️ No default weights since we are loading our own
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)

model.load_state_dict(torch.load('best_model_dummy.pth', map_location=device))  # ⬅️ Load from given path
model = model.to(device)
model.eval() 

# Freeze all layers except last few (⬅️ Same)
for param in model.parameters():
    param.requires_grad = False
for param in model.features[-3:].parameters():
    param.requires_grad = True
for param in model.classifier.parameters():
    param.requires_grad = True

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

C:\Users\Asus\AppData\Local\Temp\ipykernel_16944\1553475832.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_dummy.pth', map_

In [ ]:
# Training loop with early stopping (⬅️ Added Early Stopping logic)
best_val_acc = 0
patience = 3
counter = 0

for epoch in range(20):
    print(f"Epoch {epoch} started.")
    model.train()
    correct = 0
    total = 0
    running_loss = 0.0
    cnt=0
    for images, labels in train_loader:
        cnt+=1
        print(f"Epoch {epoch}, count: {cnt}")
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    print("  Evaluating the model")
    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}: Train Acc = {train_acc:.2f}%, Val Acc = {val_acc:.2f}%")
    scheduler.step()

    # Free up unused memory (⬅️ Added)
    torch.cuda.empty_cache()
    torch.save(model.state_dict(), f'model__{epoch}_{val_acc}_{train_acc}.pth')

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("⛔ Early stopping triggered.")
            break


Epoch 0 started.


In [ ]:


# Load best model for inference
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Inference on test data
preds = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        preds.extend(predicted.cpu().numpy())

# Submission
submission = pd.DataFrame({
    'id': test_df['file_name'].apply(lambda x: os.path.basename(x)),
    'label': preds
})
submission.to_csv('final_submission.csv', index=False)
print("✅ final_submission.csv saved successfully!")
print(submission.head())


In [9]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Build dataset from folders
def build_dataframe():
    data = []
    for label, folder in enumerate(['ai', 'real']):
        n = 0
        for file in os.listdir(folder):
            if n == 10000:
                break
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.webp', '.gif')):
                n += 1
                data.append({'file_name': os.path.join(folder, file), 'label': label})
    df = pd.DataFrame(data)
    return train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

train_df, val_df = build_dataframe()
test_files = os.listdir('test_data/teamspace/studios/this_studio/final_test_renamed')
test_df = pd.DataFrame({'file_name': [f'test_data/teamspace/studios/this_studio/final_test_renamed/{x}' for x in test_files]})

# Dataset
class ImageDataset(Dataset):
    def __init__(self, df, transform=None, is_test=False):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.iloc[idx]['file_name']
        try:
            image = Image.open(path).convert('RGB')
        except (OSError, ValueError) as e:
            print(f"⚠️ Skipping corrupted image: {path}")
            # Fallback: Return a black image
            image = Image.new('RGB', (224, 224), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image, -1
        else:
            label = int(self.df.iloc[idx]['label'])
            return image, label


# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Hyperparameters from Optuna
BATCH_SIZE = 64
LR = 0.000762
WEIGHT_DECAY = 0.00038036

# DataLoaders (⬅️ Made memory-efficient: num_workers=2 and pin_memory=True)
train_dataset = ImageDataset(train_df, transform=train_transform)
val_dataset = ImageDataset(val_df, transform=test_transform)
test_dataset = ImageDataset(test_df, transform=test_transform, is_test=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

# Load pretrained model from saved weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = efficientnet_b0(weights=None)  # ⬅️ No default weights since we are loading our own
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)

model.load_state_dict(torch.load('best_model_2.pth', map_location=device))  # ⬅️ Load from given path
model = model.to(device)
model.eval()  # Set to eval mode for inference


# Freeze all layers except last few (⬅️ Same)
for param in model.parameters():
    param.requires_grad = False
for param in model.features[-3:].parameters():
    param.requires_grad = True
for param in model.classifier.parameters():
    param.requires_grad = True

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop with early stopping (⬅️ Added Early Stopping logic)
best_val_acc = 0
patience = 3
counter = 0

for epoch in range(20):
    print(f"Epch no {epoch}")
    model.train()
    correct = 0
    total = 0
    running_loss = 0.0
    n =0
    for images, labels in train_loader:
        n+=1
        print(f"epoch {epoch } sub ep {n}")
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    torch.save(model.state_dict(), f'best_model_3_epoch {epoch}_train_{train_acc}_val_{val_acc}.pth')
    print(f"Epoch {epoch+1}: Train Acc = {train_acc:.2f}%, Val Acc = {val_acc:.2f}%")
    scheduler.step()

    # Free up unused memory (⬅️ Added)
    torch.cuda.empty_cache()

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model_3.pth')
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("⛔ Early stopping triggered.")
            break

# Load best model for inference
model.load_state_dict(torch.load('best_model_3.pth'))
model.eval()

# Inference on test data
preds = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        preds.extend(predicted.cpu().numpy())

# Submission
submission = pd.DataFrame({
    'filename': test_df['file_name'].apply(lambda x: os.path.basename(x)),
    'class': preds
})
submission.to_csv('final_submission_3.csv', index=False)
print("✅ final_submission.csv saved successfully!")
print(submission.head())


C:\Users\Asus\AppData\Local\Temp\ipykernel_15336\924277021.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_2.pth', map_loca

Epch no 0
epoch 0 sub ep 1
epoch 0 sub ep 2
epoch 0 sub ep 3
epoch 0 sub ep 4
epoch 0 sub ep 5
epoch 0 sub ep 6
epoch 0 sub ep 7
epoch 0 sub ep 8
epoch 0 sub ep 9
epoch 0 sub ep 10
epoch 0 sub ep 11
epoch 0 sub ep 12
epoch 0 sub ep 13
epoch 0 sub ep 14
epoch 0 sub ep 15
epoch 0 sub ep 16
epoch 0 sub ep 17
epoch 0 sub ep 18
epoch 0 sub ep 19
epoch 0 sub ep 20
epoch 0 sub ep 21
epoch 0 sub ep 22
epoch 0 sub ep 23
epoch 0 sub ep 24
epoch 0 sub ep 25
epoch 0 sub ep 26
epoch 0 sub ep 27
epoch 0 sub ep 28
epoch 0 sub ep 29
epoch 0 sub ep 30
epoch 0 sub ep 31
epoch 0 sub ep 32
epoch 0 sub ep 33
epoch 0 sub ep 34
epoch 0 sub ep 35
epoch 0 sub ep 36
epoch 0 sub ep 37
epoch 0 sub ep 38
epoch 0 sub ep 39
epoch 0 sub ep 40
epoch 0 sub ep 41
epoch 0 sub ep 42
epoch 0 sub ep 43
epoch 0 sub ep 44
epoch 0 sub ep 45
epoch 0 sub ep 46
epoch 0 sub ep 47
epoch 0 sub ep 48
epoch 0 sub ep 49
epoch 0 sub ep 50
epoch 0 sub ep 51
epoch 0 sub ep 52
epoch 0 sub ep 53
epoch 0 sub ep 54
epoch 0 sub ep 55
epoch 0 s

C:\Users\Asus\AppData\Local\Temp\ipykernel_15336\924277021.py:177: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_3.pth'))
c:\Use

✅ final_submission.csv saved successfully!
   filename  class
0     1.jpg      0
1    10.jpg      1
2   100.jpg      1
3  1000.jpg      0
4  1001.jpg      1


In [10]:
model.load_state_dict(torch.load('best_model_3_epoch 2_train_95.3375_val_93.95.pth'))
model.eval()

# Inference on test data
preds = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        preds.extend(predicted.cpu().numpy())

# Submission
submission = pd.DataFrame({
    'filename': test_df['file_name'].apply(lambda x: os.path.basename(x)),
    'class': preds
})
submission.to_csv('final_submission_3_1.csv', index=False)
print("✅ final_submission.csv saved successfully!")
print(submission.head())

C:\Users\Asus\AppData\Local\Temp\ipykernel_15336\19415205.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_3_epoch 2_train_95

✅ final_submission.csv saved successfully!
   filename  class
0     1.jpg      0
1    10.jpg      1
2   100.jpg      1
3  1000.jpg      0
4  1001.jpg      1


## Changed the Batch size to 128 and dataset size = 30000

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Build dataset from folders
def build_dataframe():
    data = []
    for label, folder in enumerate(['ai', 'real']):
        n = 0
        for file in os.listdir(folder):
            if n == 30000:
                break
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.webp', '.gif')):
                n += 1
                data.append({'file_name': os.path.join(folder, file), 'label': label})
    df = pd.DataFrame(data)
    return train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

train_df, val_df = build_dataframe()
test_files = os.listdir('test_data/teamspace/studios/this_studio/final_test_renamed')
test_df = pd.DataFrame({'file_name': [f'test_data/teamspace/studios/this_studio/final_test_renamed/{x}' for x in test_files]})

# Dataset
class ImageDataset(Dataset):
    def __init__(self, df, transform=None, is_test=False):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.iloc[idx]['file_name']
        try:
            image = Image.open(path).convert('RGB')
        except (OSError, ValueError) as e:
            print(f"⚠️ Skipping corrupted image: {path}")
            # Fallback: Return a black image
            image = Image.new('RGB', (224, 224), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image, -1
        else:
            label = int(self.df.iloc[idx]['label'])
            return image, label


# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Hyperparameters from Optuna
BATCH_SIZE = 128
LR = 0.000762
WEIGHT_DECAY = 0.00038036

# DataLoaders (⬅️ Made memory-efficient: num_workers=2 and pin_memory=True)
train_dataset = ImageDataset(train_df, transform=train_transform)
val_dataset = ImageDataset(val_df, transform=test_transform)
test_dataset = ImageDataset(test_df, transform=test_transform, is_test=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

# Load pretrained model from saved weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = efficientnet_b0(weights=None)  # ⬅️ No default weights since we are loading our own
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)

model.load_state_dict(torch.load('best_model_3.pth', map_location=device))  # ⬅️ Load from given path
model = model.to(device)
model.eval()  # Set to eval mode for inference


# Freeze all layers except last few (⬅️ Same)
for param in model.parameters():
    param.requires_grad = False
for param in model.features[-3:].parameters():
    param.requires_grad = True
for param in model.classifier.parameters():
    param.requires_grad = True

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop with early stopping (⬅️ Added Early Stopping logic)
best_val_acc = 0
patience = 3
counter = 0

for epoch in range(20):
    print(f"Epch no {epoch}")
    model.train()
    correct = 0
    total = 0
    running_loss = 0.0
    n =0
    for images, labels in train_loader:
        n+=1
        print(f"epoch {epoch } sub ep {n}")
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    torch.save(model.state_dict(), f'best_model_4_epoch {epoch}_train_{train_acc}_val_{val_acc}.pth')
    print(f"Epoch {epoch+1}: Train Acc = {train_acc:.2f}%, Val Acc = {val_acc:.2f}%")
    scheduler.step()

    # Free up unused memory (⬅️ Added)
    torch.cuda.empty_cache()

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model_4.pth')
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("⛔ Early stopping triggered.")
            break

# Load best model for inference
model.load_state_dict(torch.load('best_model_4.pth'))
model.eval()

# Inference on test data
preds = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        preds.extend(predicted.cpu().numpy())

# Submission
submission = pd.DataFrame({
    'filename': test_df['file_name'].apply(lambda x: os.path.basename(x)),
    'class': preds
})
submission.to_csv('final_submission_4.csv', index=False)
print("✅ final_submission.csv saved successfully!")
print(submission.head())

C:\Users\Asus\AppData\Local\Temp\ipykernel_15336\2235403929.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_3.pth', map_loc

In [ ]:
# Load best model for inference
model.load_state_dict(torch.load('best_model_4_epoch 10_train_98.6387198557584_val_91.76958442260886.pth'))
model.eval()

# Inference on test data
preds = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        preds.extend(predicted.cpu().numpy())

# Submission
submission = pd.DataFrame({
    'id': test_df['file_name'].apply(lambda x: os.path.basename(x)),
    'label': preds
})
submission.to_csv('final_submission_4_5.csv', index=False)
print("✅ final_submission.csv saved successfully!")
print(submission.head())


C:\Users\Asus\AppData\Local\Temp\ipykernel_16944\190736668.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_4_epoch 10_train_